# 内容一：在MNIST数据集上构建网络进行分类

## 1. 实验前导

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data as tud
import numpy as np

## 2. 准备数据

### 学会使用Dataloader来加载数据
Dataloader能够帮我们打乱数据集，拿到batch数据 \
为了使用Dataloader，需要定义以下三个function
- \__init__: 模型初始化
- \__len__: 返回整个数据集有多少item
- \__getitem__: 根据给定的index返回一个item

调用Dataloader之前还要先定义dataset

In [2]:
# Pytorch帮助我们预先加载了一些常用的数据集
# 如果使用这些数据集，会相对容易的进行数据加载
# 例如：常用的Mnist数据集
mnist_train_data = datasets.MNIST("./data",train=True,download=True,
                                 transform = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=(0.13066062,),std=(0.30810776,))
                                 ]))
batch_size = 64
train_dataloader = tud.DataLoader(mnist_train_data,batch_size = batch_size,shuffle=True) # 将dataset转换为iterator
mnist_test_data = datasets.MNIST("./data",train=False,download=True,
                                 transform = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=(0.13066062,),std=(0.30810776,))
                                 ]))
test_dataloader = tud.DataLoader(mnist_test_data,batch_size = batch_size)

## 3. 配置网络

### (1) 定义网络
- 继承 nn.Module
- 初始化函数
- forward 函数
- 其余可以根据模型需要定义相关的函数

In [3]:
# 定义一个简单的基于ConvNet的简单神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__() # the input is 1*28*28
        self.conv1 = nn.Conv2d(1,20,5,1) # (28-5)/1+1=24, 20*24*24
        self.conv2 = nn.Conv2d(20,50,5,1) # 12-5+1=8
        self.fc1 = nn.Linear(4*4*50,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        x = F.relu(self.conv1(x)) # 20 * 24 * 24
        x = F.max_pool2d(x,2,2) # 20 * 12 * 12
        x = F.relu(self.conv2(x)) # 50 * 8 * 8
        x = F.max_pool2d(x,2,2) # 50 * 4 * 4
        x = x.view(-1,4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x #F.log_softmax(x,dim=1)
model = Net()

### (2) 定义损失函数

In [4]:
loss_fn = nn.CrossEntropyLoss(reduction='mean')

### (3) 定义优化算法

In [5]:
lr = 0.01
momentum = 0.5
optimizer = optim.SGD(model.parameters(),lr=lr,momentum=momentum)

## 4. 训练网络

- 模型一般需要训练若干个epoch
- 每个epoch我们把所有数据分成若干个batch
- 把每个batch的输入和输出都包装成cuda Tensor
- forward pass
- 计算loss
- 清空gradient
- backward pass
- 更新模型参数
- 每隔一定的iteration输出loss，以及在验证集上做模型的评估

In [6]:
def train(model,train_dataloader,loss_fn,optimizer,epoch):
    model.train()
    for idx, (data, label) in enumerate(train_dataloader):
        output = model(data)
        loss = loss_fn(output,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if idx % 100 == 0:
            print("Train Epoch: {}, iteration: {}, loss: {}".format(
                epoch,idx,loss.item()))  
    return model

## 5. 模型评估

In [7]:
def test(model,test_dataloader,loss_fn):
    model.eval()
    total_loss = 0.
    correct = 0.
    with torch.no_grad():
        for idx, (data,target) in enumerate(test_dataloader):
            output = model(data) # batch_size * 10        
            loss = loss_fn(output,target)*output.size(0)
            pred = output.argmax(dim=1)
            total_loss += loss
            correct += pred.eq(target).sum()
    total_loss /= len(test_dataloader.dataset)
    acc = 100.*correct/len(test_dataloader.dataset)
    print("Test Loss:{}, Accuracy:{}".format(total_loss,acc))
    return acc

In [8]:
num_epochs = 2
for epoch in range(num_epochs):
    model = train(model,train_dataloader,loss_fn,optimizer,epoch)
    test(model,test_dataloader,loss_fn)

Train Epoch: 0, iteration: 0, loss: 2.307435989379883
Train Epoch: 0, iteration: 100, loss: 0.5602225065231323
Train Epoch: 0, iteration: 200, loss: 0.19598789513111115
Train Epoch: 0, iteration: 300, loss: 0.23787333071231842
Train Epoch: 0, iteration: 400, loss: 0.08825021237134933
Train Epoch: 0, iteration: 500, loss: 0.2534177601337433
Train Epoch: 0, iteration: 600, loss: 0.170552060008049
Train Epoch: 0, iteration: 700, loss: 0.1840806007385254
Train Epoch: 0, iteration: 800, loss: 0.12317674607038498
Train Epoch: 0, iteration: 900, loss: 0.06666550040245056
Test Loss:0.08838915824890137, Accuracy:97
Train Epoch: 1, iteration: 0, loss: 0.0736343190073967
Train Epoch: 1, iteration: 100, loss: 0.10537996143102646
Train Epoch: 1, iteration: 200, loss: 0.03152836114168167
Train Epoch: 1, iteration: 300, loss: 0.07204625755548477
Train Epoch: 1, iteration: 400, loss: 0.13851375877857208
Train Epoch: 1, iteration: 500, loss: 0.04532371088862419
Train Epoch: 1, iteration: 600, loss: 0.0

## 6. 模型存储

In [9]:
#torch.save(model.state_dict(),"mnist_cnn.pth")
# num_epochs = 2
best_valid_acc = 0.
for epoch in range(num_epochs):
    train(model,train_dataloader,loss_fn,optimizer,epoch)
    acc = test(model,test_dataloader,loss_fn)
    if acc > best_valid_acc:
        best_valid_acc = acc
        torch.save(model.state_dict(),"best_mnist_cnn.pth")

Train Epoch: 0, iteration: 0, loss: 0.027293963357806206
Train Epoch: 0, iteration: 100, loss: 0.15356257557868958
Train Epoch: 0, iteration: 200, loss: 0.0734483003616333
Train Epoch: 0, iteration: 300, loss: 0.043793752789497375
Train Epoch: 0, iteration: 400, loss: 0.08779671043157578
Train Epoch: 0, iteration: 500, loss: 0.016259178519248962
Train Epoch: 0, iteration: 600, loss: 0.03868456929922104
Train Epoch: 0, iteration: 700, loss: 0.011153661645948887
Train Epoch: 0, iteration: 800, loss: 0.028804047033190727
Train Epoch: 0, iteration: 900, loss: 0.07739989459514618
Test Loss:0.043091725558042526, Accuracy:98
Train Epoch: 1, iteration: 0, loss: 0.11538230627775192
Train Epoch: 1, iteration: 100, loss: 0.016172412782907486
Train Epoch: 1, iteration: 200, loss: 0.013750119134783745
Train Epoch: 1, iteration: 300, loss: 0.009204431436955929
Train Epoch: 1, iteration: 400, loss: 0.009066282771527767
Train Epoch: 1, iteration: 500, loss: 0.10174877941608429
Train Epoch: 1, iteratio

### Load模型

In [10]:
test_model = Net()
test_model.load_state_dict(torch.load("mnist_cnn.pth"))
test(model,test_dataloader,loss_fn)

Test Loss:0.03979605808854103, Accuracy:98


tensor(98)

### For FashionMNIST

In [11]:
batch_size = 32
train_dataloader = tud.DataLoader(
    datasets.FashionMNIST("./fashion_mnist_data",train=True,download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.2860402,),std=(0.3530239,))
                   ])),
    batch_size=batch_size,
    shuffle=True) # 将dataset转换为iterator
test_dataloader = tud.DataLoader(
    datasets.FashionMNIST("./fashion_mnist_data",train=False,download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.2860402,),std=(0.3530239,))
                   ])),
    batch_size=batch_size) # 将dataset转换为iterator
lr = 0.01
momentum = 0.5
model = Net()
optimizer = optim.SGD(model.parameters(),lr=lr,momentum=momentum)
num_epochs = 2

for epoch in range(num_epochs):
    train(model,train_dataloader,loss_fn,optimizer,epoch)
    test(model,test_dataloader,loss_fn)
    
torch.save(model.state_dict(),"fashion_mnist_cnn.pth")

Train Epoch: 0, iteration: 0, loss: 2.295435905456543
Train Epoch: 0, iteration: 100, loss: 1.2099932432174683
Train Epoch: 0, iteration: 200, loss: 0.9759495854377747
Train Epoch: 0, iteration: 300, loss: 0.632256031036377
Train Epoch: 0, iteration: 400, loss: 0.7355301380157471
Train Epoch: 0, iteration: 500, loss: 0.6270371675491333
Train Epoch: 0, iteration: 600, loss: 0.6622246503829956
Train Epoch: 0, iteration: 700, loss: 0.5373417139053345
Train Epoch: 0, iteration: 800, loss: 0.7718546390533447
Train Epoch: 0, iteration: 900, loss: 0.4907580614089966
Train Epoch: 0, iteration: 1000, loss: 0.5114668011665344
Train Epoch: 0, iteration: 1100, loss: 0.5800931453704834
Train Epoch: 0, iteration: 1200, loss: 0.6064159870147705
Train Epoch: 0, iteration: 1300, loss: 0.5081970691680908
Train Epoch: 0, iteration: 1400, loss: 0.4496583938598633
Train Epoch: 0, iteration: 1500, loss: 0.26464536786079407
Train Epoch: 0, iteration: 1600, loss: 0.7572592496871948
Train Epoch: 0, iteration: 

# 内容二：CNN模型的迁移学习

- 很多时候当我们训练一个新的图像分类任务，我们不会完全从一个随机的模型开始训练，而是利用预训练的模型来加速训练的过程。我们经常使用在ImageNet上的预训练模型
- 有两种方法做迁移学习
    - finetuning：从一个预训练模型开始，改变一些模型的架构，然后继续训练整个模型的参数；
    - feature extraction：不改变预训练模型的参数，只更新我们改变过的部分模型参数。（当成特征提取器来使用）

## 1. 实验前导

In [12]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets, transforms
import torch.utils.data as tud
import numpy as np

## 2. 准备数据

数据：使用hymenoptera_data数据集 \
数据集包括两类图片，bees和ants。这些数据都被处理成了可以使用ImageFolder来读取的格式。我们只需要把data_dir设置成数据的根目录，然后把model_name设置成我们想要使用的预训练模型

In [13]:
# format
data_dir = "./data/hymenoptera_data"
model_name = "resnet18"
num_class = 2
#feature_extract = True
input_size = 224

读入数据: 把数据预处理成相应的格式

In [14]:
# progress
all_imgs = datasets.ImageFolder(os.path.join(data_dir,"train"),
                                transform=transforms.Compose([
                                    transforms.RandomResizedCrop(input_size),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),                                    
                                ]))
loader = tud.DataLoader(all_imgs,batch_size=batch_size,shuffle=True)

In [15]:
all_imgs[20][0]

tensor([[[0.7294, 0.7255, 0.7020,  ..., 0.4667, 0.4863, 0.4980],
         [0.7137, 0.7059, 0.6863,  ..., 0.4706, 0.4745, 0.4863],
         [0.7137, 0.7020, 0.6784,  ..., 0.4706, 0.4824, 0.4941],
         ...,
         [0.5412, 0.4392, 0.6118,  ..., 0.4980, 0.4510, 0.3843],
         [0.5373, 0.4627, 0.4980,  ..., 0.4863, 0.4980, 0.4588],
         [0.5922, 0.5020, 0.4314,  ..., 0.4941, 0.4824, 0.4471]],

        [[0.7216, 0.7137, 0.6941,  ..., 0.4784, 0.4902, 0.5020],
         [0.7137, 0.7059, 0.6863,  ..., 0.4784, 0.4863, 0.4980],
         [0.7176, 0.7020, 0.6784,  ..., 0.4824, 0.4863, 0.4980],
         ...,
         [0.5529, 0.4510, 0.6471,  ..., 0.5216, 0.4941, 0.4078],
         [0.5490, 0.4745, 0.5216,  ..., 0.5216, 0.5294, 0.4941],
         [0.6196, 0.5137, 0.4549,  ..., 0.5451, 0.5333, 0.4706]],

        [[0.7333, 0.7176, 0.6941,  ..., 0.5020, 0.5176, 0.5176],
         [0.7333, 0.7176, 0.6980,  ..., 0.4980, 0.5176, 0.5176],
         [0.7412, 0.7137, 0.7020,  ..., 0.5176, 0.5216, 0.

In [16]:
# format
batch_size = 32
train_imgs = datasets.ImageFolder(os.path.join(data_dir,"train"),
                                transform=transforms.Compose([
                                    transforms.RandomResizedCrop(input_size),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229,0.224,0.225])
                                ]))
train_dataloader = tud.DataLoader(train_imgs,batch_size=batch_size,shuffle=True)
test_imgs = datasets.ImageFolder(os.path.join(data_dir,"val"),
                                transform=transforms.Compose([
                                    transforms.Resize(input_size),  
                                    transforms.CenterCrop(input_size),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229,0.224,0.225])
                                ]))
test_dataloader = tud.DataLoader(test_imgs,batch_size=batch_size)

## 3. 配置网络

### (1) 定义网络

In [17]:
# format
def initialize_model(model_name,num_class,use_pretrained=True,feature_extract=True):
    if model_name == "resnet18":
        model_ft = models.resnet18(pretrained=use_pretrained)
        if feature_extract: # do not update the parameters
            for param in model_ft.parameters():
                param.requires_grad = False
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_class)        
    else:
        print("model not implemented")
        return None
    return model_ft
model_ft = initialize_model("resnet18",2,use_pretrained=False,feature_extract=False)

In [18]:
print(model_ft.layer1[0].conv1.weight.requires_grad)
print(model_ft.fc.weight.requires_grad)

True
True


### (2) 定义损失函数

In [19]:
loss_fn = nn.CrossEntropyLoss()

### (3) 定义优化算法

In [20]:
lr = 0.01
momentum = 0.5
optimizer = optim.SGD(model_ft.parameters(),lr=lr,momentum=momentum)

## 4. 训练网络

In [21]:
def train_model(model,train_dataloader,loss_fn,optimizer,epoch):
    model.train()
    total_loss = 0.
    total_corrects = 0.
    for idx, (inputs, labels) in enumerate(train_dataloader):
        outputs = model(inputs)
        loss = loss_fn(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        preds = outputs.argmax(dim=1)
        total_loss += loss.item() * inputs.size(0)
        total_corrects += torch.sum(preds.eq(labels))
    epoch_loss = total_loss / len(train_dataloader.dataset)
    epoch_accuracy = 100.*total_corrects / len(train_dataloader.dataset)
    print("Epoch:{}, Training Loss:{}, Traning Acc:{}".format(epoch,epoch_loss,epoch_accuracy))  
    #return model        

## 5. 模型评估

In [22]:
def test_model(model,test_dataloader,loss_fn):
    model.eval()
    total_loss = 0.
    total_corrects = 0.
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(test_dataloader):
            outputs = model(inputs)
            loss = loss_fn(outputs,labels)
            preds = outputs.argmax(dim=1)
            total_loss += loss.item() * inputs.size(0)
            total_corrects += torch.sum(preds.eq(labels))
    epoch_loss = total_loss / len(test_dataloader.dataset)
    epoch_accuracy = 100.*total_corrects / len(test_dataloader.dataset)
    print("acc type:", epoch_accuracy)
    print("Test Loss:{}, Test Acc:{}".format(epoch_loss,epoch_accuracy))  
    return epoch_accuracy 

In [23]:
num_epochs = 5
for epoch in range(num_epochs):
    train_model(model_ft,train_dataloader,loss_fn,optimizer,epoch)
    acc = test_model(model_ft,test_dataloader,loss_fn)

Epoch:0, Training Loss:0.7031608497510191, Traning Acc:58
acc type: tensor(50)
Test Loss:0.7017492550650454, Test Acc:50
Epoch:1, Training Loss:0.6760420105496391, Traning Acc:59
acc type: tensor(55)
Test Loss:0.6705557723450505, Test Acc:55
Epoch:2, Training Loss:0.6359495246996645, Traning Acc:65
acc type: tensor(48)
Test Loss:0.8494572016148786, Test Acc:48
Epoch:3, Training Loss:0.7302298384611724, Traning Acc:57
acc type: tensor(69)
Test Loss:0.6352221517781027, Test Acc:69
Epoch:4, Training Loss:0.6081345179041878, Traning Acc:67
acc type: tensor(69)
Test Loss:0.582625621868894, Test Acc:69


# 扩展：resnet50，冻结某些层参数不训练

冻结前7层

In [24]:
model_ft = models.resnet50(pretrained=True)
ct = 0
for child in model_ft.children():
    ct += 1
    if ct < 7:
        for param in child.parameters():
            param.requires_grad = False

In [25]:
# 优化器中filter一下
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model_ft.parameters()), lr=1e-3)

## 冻结某些层参数不训练

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    train_model(model_ft,train_dataloader,loss_fn,optimizer,epoch)
    acc = test_model(model_ft,test_dataloader,loss_fn)

Epoch:0, Training Loss:8.682191676780826, Traning Acc:0
acc type: tensor(0)
Test Loss:9.149730897417255, Test Acc:0
Epoch:1, Training Loss:6.392732839115331, Traning Acc:4
acc type: tensor(5)
Test Loss:5.709677166408962, Test Acc:5
Epoch:2, Training Loss:4.406351249726092, Traning Acc:15
acc type: tensor(25)
Test Loss:3.658740886675766, Test Acc:25
Epoch:3, Training Loss:2.7739955440896455, Traning Acc:43
acc type: tensor(52)
Test Loss:2.2434550001730327, Test Acc:52
